In [5]:
# import library
import pandas as pd # for data manipulation

%pip install pyodide-http
import pyodide_http # for loading external files from http 
pyodide_http.patch_all() 

import matplotlib.pyplot as plt # for plottings 

# import data 
weather = pd.read_csv('https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/wetter.csv')
kiwo = pd.read_csv('https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/kiwo.csv')
sale = pd.read_csv('https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/umsatzdaten_gekuerzt.csv')

# merge data 
sale_weather_kiwo = sale.merge(weather, on='Datum', how='outer', suffixes = ("_sale", "_weather")) \
.merge(kiwo, on='Datum', how='outer', suffixes = ("", "_kiwo"))

sale_weather_kiwo.head()


280.62s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


,id,Datum,Warengruppe,Umsatz,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,KielerWoche
0,NaN,2012-01-01,NaN,NaN,8.0,9.8250,14.0,58.0,NaN
1,NaN,2012-01-02,NaN,NaN,7.0,7.4375,12.0,NaN,NaN
2,NaN,2012-01-03,NaN,NaN,8.0,5.5375,18.0,63.0,NaN
3,NaN,2012-01-04,NaN,NaN,4.0,5.6875,19.0,80.0,NaN
4,NaN,2012-01-05,NaN,NaN,6.0,5.3000,23.0,80.0,NaN
